# Basic

In [0]:
from pylab import *
from numpy import *
from scipy import linalg


def compute_fundamental(x1,x2):
    """    Computes the fundamental matrix from corresponding points 
        (x1,x2 3*n arrays) using the 8 point algorithm.
        Each row in the A matrix below is constructed as
        [x'*x, x'*y, x', y'*x, y'*y, y', x, y, 1] """
    
    n = x1.shape[1]
    if x2.shape[1] != n:
        raise ValueError("Number of points don't match.")
    
    # build matrix for equations
    A = zeros((n,9))
    for i in range(n):
        A[i] = [x1[0,i]*x2[0,i], x1[0,i]*x2[1,i], x1[0,i]*x2[2,i],
                x1[1,i]*x2[0,i], x1[1,i]*x2[1,i], x1[1,i]*x2[2,i],
                x1[2,i]*x2[0,i], x1[2,i]*x2[1,i], x1[2,i]*x2[2,i] ]
            
    # compute linear least square solution
    U,S,V = linalg.svd(A)
    F = V[-1].reshape(3,3)
        
    # constrain F
    # make rank 2 by zeroing out last singular value
    U,S,V = linalg.svd(F)
    S[2] = 0
    F = dot(U,dot(diag(S),V))
    
    return F/F[2,2]


def compute_epipole(F):
    """ Computes the (right) epipole from a 
        fundamental matrix F. 
        (Use with F.T for left epipole.) """
    
    # return null space of F (Fx=0)
    U,S,V = linalg.svd(F)
    e = V[-1]
    return e/e[2]
    
    
def plot_epipolar_line(im,F,x,epipole=None,show_epipole=True):
    """ Plot the epipole and epipolar line F*x=0
        in an image. F is the fundamental matrix 
        and x a point in the other image."""
    
    m,n = im.shape[:2]
    line = dot(F,x)
    
    # epipolar line parameter and values
    t = linspace(0,n,100)
    lt = array([(line[2]+line[0]*tt)/(-line[1]) for tt in t])

    # take only line points inside the image
    ndx = (lt>=0) & (lt<m) 
    plot(t[ndx],lt[ndx],linewidth=2)
    
    if show_epipole:
        if epipole is None:
            epipole = compute_epipole(F)
        plot(epipole[0]/epipole[2],epipole[1]/epipole[2],'r*')
    



def compute_fundamental_normalized(x1,x2):
    """    Computes the fundamental matrix from corresponding points 
        (x1,x2 3*n arrays) using the normalized 8 point algorithm. """

    n = x1.shape[1]
    if x2.shape[1] != n:
        raise ValueError("Number of points don't match.")

    # normalize image coordinates
    x1 = x1 / x1[2]
    mean_1 = mean(x1[:2],axis=1)
    S1 = sqrt(2) / std(x1[:2])
    T1 = array([[S1,0,-S1*mean_1[0]],[0,S1,-S1*mean_1[1]],[0,0,1]])
    x1 = dot(T1,x1)
    
    x2 = x2 / x2[2]
    mean_2 = mean(x2[:2],axis=1)
    S2 = sqrt(2) / std(x2[:2])
    T2 = array([[S2,0,-S2*mean_2[0]],[0,S2,-S2*mean_2[1]],[0,0,1]])
    x2 = dot(T2,x2)

    # compute F with the normalized coordinates
    F = compute_fundamental(x1,x2)
    #print (F)
    # reverse normalization
    F = dot(T1.T,dot(F,T2))

    return F/F[2,2]





# IMPR

In [0]:
from scipy.signal import convolve2d
import numpy as np

import scipy.io.wavfile
from scipy.ndimage.filters import convolve
import matplotlib.image as mpimg
from skimage.color import rgb2gray

RGB = 2
GRAY = 1

def gaussian_kernel(kernel_size):
    conv_kernel = np.array([1, 1], dtype=np.float64)[:, None]
    conv_kernel = convolve2d(conv_kernel, conv_kernel.T)
    kernel = np.array([1], dtype=np.float64)[:, None]
    for i in range(kernel_size - 1):
        kernel = convolve2d(kernel, conv_kernel, 'full')
    return kernel / kernel.sum()


def blur_spatial(img, kernel_size):
    kernel = gaussian_kernel(kernel_size)
    blur_img = np.zeros_like(img)
    if len(img.shape) == 2:
        blur_img = convolve2d(img, kernel, 'same', 'symm')
    else:
        for i in range(3):
            blur_img[..., i] = convolve2d(img[..., i], kernel, 'same', 'symm')
    return blur_img


def read_image(file_name, representation):
    """reads an image file and converts it into a given representation.
        defning whether the output should be a grayscale image (1) or an RGB image (2).
    """
    img = mpimg.imread(file_name).astype(np.float64)
    if representation == GRAY:
        return rgb2gray(img) / 255
    elif representation == RGB:
        return img / 255
    else:
        print("ERROR")

def build_gaussian_pyramid(im, max_levels, filter_size):
    """
    :param im:  a grayscale image with double values in [0, 1]
    :param max_levels: the maximal number of levels1 in the resulting pyramid.
    :param filter_size: – the size of the Gaussian filter (an odd scalar that represents a squared filter) to be used
            in constructing the pyramid filter (e.g for filter_size = 3 you should get [0.25, 0.5, 0.25]).
    :return: The resulting pyramid pyr as a standard python array (i.e. not numpy’s array)
            with maximum length of max_levels, where each element of the array is a grayscale image.
            And a filter_vec which is row vector of shape (1, filter_size) used for the pyramid construction.
            This filter should be built using a consequent 1D convolutions of [1 1] with itself in order to derive a
            row of the binomial coefficients which is a good approximation to the Gaussian profile.
            The filter_vec should be normalized.
    """
    pyr = []
    pyr.append(im)
    filter_vec = np.array([1, 1])
    conv_vec = np.array([1, 1])
    for i in range(1, filter_size - 1):
        filter_vec = scipy.signal.convolve(filter_vec, conv_vec)
        filter_vec = filter_vec / filter_vec.sum()
    filter_vec = filter_vec.astype(np.float64)
    filter_vec = filter_vec[np.newaxis, :]
    for i in range(max_levels - 1):
        # on reduce operations you should convolve with filter_vec twice - once as a row vector and then as a column vector
        blured = convolve(im, filter_vec)
        blured_tran = convolve(blured, filter_vec.T)
        # down-sample an image by taking its even indexes
        im = blured_tran[::2, ::2]
        # the minimum dimension (height or width) of the lowest resolution image in the pyramid is not smaller than 16
        if (im.shape[0] < 16 or im.shape[1] < 16):
            break
        pyr.append(im.astype(np.float64))
    return pyr, filter_vec

In [16]:
import numpy as np
import os
import matplotlib.pyplot as plt

from scipy.ndimage.morphology import generate_binary_structure
from scipy.ndimage.filters import maximum_filter
from scipy.ndimage import label, center_of_mass
import shutil
from imageio import imwrite
# import sol4_utils

import matplotlib.image as mpimg
from skimage.color import rgb2gray
import scipy.io.wavfile
from scipy import signal
from scipy.ndimage.filters import convolve

from scipy import ndimage
import random


# ========================= ex4 =======================


def harris_corner_detector(im):
    """
    Detects harris corners.
    Make sure the returned coordinates are x major!!!
    :param im: A 2D array representing a grayscale image.
    :return: An array with shape (N,2), where ret[i,:] are the [x,y] coordinates of the ith corner points.
    """
    # Get the Ix and Iy derivatives of the image using the filters [1, 0, −1], [1, 0, −1]T respectively.
    filter_vec = np.array([[-1], [0], [1]])
    Ix = scipy.signal.convolve2d(im, filter_vec, mode='same', boundary='symm')
    Iy = scipy.signal.convolve2d(im, filter_vec.T, mode='same', boundary='symm')

    # Blur the images: Ix^2, Iy^2 IxIy. You may use your blur_spatial function from ex2 with kernel_size=3.
    blured_Ix = blur_spatial(np.power(Ix, 2), 3)
    blured_Iy = blur_spatial(np.power(Iy, 2), 3)
    blured_Ixy = blur_spatial(Ix * Iy, 3)

    # One way to measure how big are the two eigenvalues is R = det(M) − k(trace(M))^2
    # We will use k = 0.04.
    calc_matrix_det = (blured_Ix * blured_Iy) - (np.power(blured_Ixy, 2))
    calc_matrix_trace = blured_Ix + blured_Iy
    R = calc_matrix_det - 0.04 * (np.power(calc_matrix_trace, 2))
    corner_array = np.transpose(np.nonzero(non_maximum_suppression(R)))
    # The coordinate order is (x,y), as opposed to the order when indexing an array which is (row, column)
    return corner_array[:, ::-1]


def sample_descriptor(im, pos, desc_rad):
    """
    Samples descriptors at the given corners.
    :param im: A 2D array representing an image.
    :param pos: An array with shape (N,2), where pos[i,:] are the [x,y] coordinates of the ith corner point.
    :param desc_rad: "Radius" of descriptors to compute.
    :return: A 3D array with shape (N,K,K) containing the ith descriptor at desc[i,:,:].
    """
    # transform point coordinates between any two pyramid levels li and lj in the following way
    # plj = (xlj, ylj) = 2^(li−lj)pli = 2^(li−lj)(xli, yli)
    pos = pos * 0.25
    patch_size = 1 + 2 * desc_rad
    grid_indices = np.indices((patch_size, patch_size))
    descriptors = []
    for cord in pos:
        x = grid_indices[0] + cord[0] - desc_rad
        y = grid_indices[1] + cord[1] - desc_rad
        coordinates = ndimage.map_coordinates(im, [y, x], order=1, prefilter=False)
        # sampled descriptor matrix, the final descriptor matrix is d = ( ˜d − µ)/|| ˜d − µ|| where µ is the mean ˜d and
        #     # ||.|| is the euclidean norm operation
        normalize = np.linalg.norm(coordinates - np.mean(coordinates))
        descriptor = (coordinates - np.mean(coordinates))
        if normalize != 0:
            descriptor = descriptor / normalize
        descriptors.append(descriptor)

    return np.float64(descriptors)


def find_features(pyr):
    """
    Detects and extracts feature points from a pyramid.
    :param pyr: Gaussian pyramid of a grayscale image having 3 levels.
    :return: A list containing:
                1) An array with shape (N,2) of [x,y] feature location per row found in the image.
                   These coordinates are provided at the pyramid level pyr[0].
                2) A feature descriptor array with shape (N,K,K)
    """
    feature_points = spread_out_corners(np.array(pyr[0]), 7, 7, 3)
    descriptor = sample_descriptor(pyr[2], feature_points, 3)
    return [feature_points, descriptor]


def match_features(desc1, desc2, min_score):
    """
    Return indices of matching descriptors.
    :param desc1: A feature descriptor array with shape (N1,K,K).
    :param desc2: A feature descriptor array with shape (N2,K,K).
    :param min_score: Minimal match score.
    :return: A list containing:
                1) An array with shape (M,) and dtype int of matching indices in desc1.
                2) An array with shape (M,) and dtype int of matching indices in desc2.
    """
    N1, M1, K1 = desc1.shape
    N2, M2, K2 = desc2.shape
    # The match score we choose between two descriptors will simply be their dot product (flattened to 1D arrays).
    flattened_desc1_1D = desc1.reshape(N1, M1 * M1)
    flattened_desc2_1D = desc2.reshape(N2, M2 * M2)
    match_score = np.dot(flattened_desc1_1D, flattened_desc2_1D.T)
    # Sj,k is in the best 2 features that match feature j in image i, from all features in image i + 1.
    # Sj,k is in the best 2 features that match feature k in image i + 1, from all features in image i.
    row_match_score_max = np.partition(match_score.T, -2)[:, -2]
    col_match_score_max = np.partition(match_score, -2)[:, -2]
    row_mat, col_mat = np.meshgrid(row_match_score_max, col_match_score_max)
    second_max_mat = np.maximum(row_mat, col_mat)
    cond = np.array(match_score >= second_max_mat)
    # • Sj,k is greater than some predefined minimal score.
    min_score_cond = np.array(match_score > min_score)
    matching_indices = np.argwhere(cond & min_score_cond)
    matching_indices_desc1 = matching_indices[:, 0]
    matching_indices_desc2 = matching_indices[:, 1]
    return [matching_indices_desc1, matching_indices_desc2]





if __name__ == "__main__":
    im1 = read_image("drive/My Drive/room1.jpeg", 1)
    im2 = read_image("drive/My Drive/room.jpeg", 1)

    pyr1, filter = build_gaussian_pyramid(im1, 3, 3)
    pyr2, filter_ = build_gaussian_pyramid(im2, 3, 3)

    p1, d1 = find_features(pyr1)
    p2, d2 = find_features(pyr2)

    match1, match2 = match_features(d1, d2, 0.5)
    pts1 = p1[match1]
    pts2 = p2[match2]

    x1 = pts1[:3,:8]
    x2 = pts2[3:6,:8]
        
    # estimate fundamental matrix and return
    F = compute_fundamental_normalized(x1,x2)
    print(F)





[[ 3.55220031e-01  3.64610826e-01 -1.22669023e+00]
 [-1.05803924e-31 -1.05996284e-31  1.29713102e-16]
 [-2.74178206e-01 -2.81426535e-01  1.00000000e+00]]


In [12]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
